In [5]:
from OFDM_Models import *
import pandas as pd

In [6]:
from keras.layers import Add, MaxPooling2D, Activation
from tensorflow.keras.layers import Input, Conv1D, Activation, BatchNormalization, Add, Dense,MaxPooling1D,Flatten,Dropout,MaxPooling2D
from tensorflow.keras.layers import Layer, Conv2D, Conv1D, Flatten, Dense, Reshape, Conv2DTranspose,Concatenate
from tensorflow import keras
from keras.regularizers import l2
from tensorflow.keras import Model, layers, regularizers

k_r = regularizers.L1L2(l1=1e-5, l2=1e-4)

def TRSN1_(input_layer):
    conv = ConvRFF(output_dim=32,kernel_size=(1,1), padding='SAME', activation="linear",trainable_scale=True,kernel_regularizer=k_r)(input_layer)
    conv1 = ConvRFF(output_dim=32, kernel_size=(3,2), padding='SAME', activation="relu",trainable_scale=True,kernel_regularizer=k_r)(conv)
    res1 = Add()([conv, conv1])
    conv2 = Conv2D(32, (3,2), padding='same', activation="relu")(res1)
    conv4 = Conv2D(32, (3,2), padding='same', activation="relu")(conv2)
    res2 = Add()([res1, conv4])
    conv5 = Conv2D(32, (3,2), padding='same', activation="relu")(res2)
    conv6 = Conv2D(32, (3,2), padding='same', activation="relu")(conv5)
    conv7 = Conv2D(32, (3,2), padding='same', activation="relu")(conv6)
    res3 = Add()([conv4, conv7])
    res4 = Add()([res3, conv])
    max_pool = MaxPooling2D(pool_size=(2,2))(res4)
    return max_pool

def TRSN(input_layer):
    conv = Conv2D(32, (1,1), padding='SAME', activation="linear")(input_layer)
    conv1 = Conv2D(32, (3,1), padding='SAME', activation="relu")(conv)
    res1 = Add()([conv, conv1])
    conv2 = Conv2D(32, (3,1), padding='same', activation="relu")(res1)
    conv4 = Conv2D(32, (3,1), padding='same', activation="relu")(conv2)
    res2 = Add()([res1, conv4])
    conv5 = Conv2D(32, (3,1), padding='same', activation="relu")(res2)
    conv6 = Conv2D(32, (3,1), padding='same', activation="relu")(conv5)
    conv7 = Conv2D(32, (3,1), padding='same', activation="relu")(conv6)
    res3 = Add()([conv4, conv7])
    res4 = Add()([res3, conv])
    max_pool = MaxPooling2D(pool_size=(2,1))(res4)
    return max_pool


In [7]:

tf.keras.backend.clear_session()
#tf.debugging.experimental.enable_dump_debug_info(
#    "./logs/debug", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1
#)
# Define input shape with complex numbers
input_shape = (512, 2,1)

# Define input layer for complex data

input_layer = Input(shape=input_shape)

TRSN1 = TRSN1_(input_layer)
TRSN2= TRSN(TRSN1)
TRSN3= TRSN(TRSN2)
TRSN4= TRSN(TRSN3)
TRSN5= TRSN(TRSN4)
TRSN6= TRSN(TRSN5)
TRSN7= TRSN(TRSN6)


flatten = Flatten()(TRSN7)

fc = Dense(128, activation='selu')(flatten)
dropout =Dropout(0.3)(fc)
output = Dense(6, activation='softmax')(dropout)

# Define the model
model = keras.Model(inputs=input_layer, outputs=output)

2024-03-04 00:36:56.439421: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 00:36:56.491010: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 00:36:56.494396: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 2, 1)]          0         []                            
                                                                                                  
 conv_rff (ConvRFF)          (None, 512, 2, 32)           65        ['input_1[0][0]']             
                                                                                                  
 conv_rff_1 (ConvRFF)        (None, 512, 2, 32)           6177      ['conv_rff[0][0]']            
                                                                                                  
 add (Add)                   (None, 512, 2, 32)           0         ['conv_rff[0][0]',            
                                                                     'conv_rff_1[0][0]']      

In [9]:
X_train = np.load("DATASETS/X_train.npy",allow_pickle=True)
y_train_onehot = np.load("DATASETS/y_train_onehot.npy",allow_pickle=True)
y_test = np.load("DATASETS/y_test.npy",allow_pickle=True)
X_test = pd.read_hdf("DATASETS/test_data.h5",key="test")

/tmp/ipykernel_16588/2731762193.py:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X_test = pd.read_hdf("DATASETS/test_data.h5",key="test")


In [10]:
X_train = X_train[:, :512, :]

In [ ]:
import tensorflow_addons as tfa

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoints/",
    save_weights_only=True,
    monitor='val_f1_score',  # Changed from 'val_acc' to 'val_accuracy'
    mode='max',
    save_best_only=True)

# EarlyStopping callback to stop training when overfitting is detected
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=25,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    restore_best_weights=True)  # Restore model weights from the epoch with the best value of the monitored quantity

model.compile(optimizer="adam",loss='categorical_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=6, average='macro')])

batch_size = 1024
epochs = 150
history = model.fit(X_train, y_train_onehot, batch_size=batch_size, epochs=epochs,
                    callbacks=[tensorboard_callback, model_checkpoint_callback, early_stopping_callback],
                    validation_split=0.3)


/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2024-03-04 00:37:03.683684: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/150


2024-03-04 00:37:10.960269: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-04 00:37:11.048720: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-04 00:37:14.650143: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff5719145f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-04 00:37:14.650182: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-04 00:37:14.656406: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709512634.763767   16636 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


171/171 [==============================] - 69s 295ms/step - loss: 2.4998 - f1_score: 0.1722 - val_loss: 2.4609 - val_f1_score: 0.1206
Epoch 2/150
171/171 [==============================] - 46s 272ms/step - loss: 1.8419 - f1_score: 0.5076 - val_loss: 1.1727 - val_f1_score: 0.7968
Epoch 3/150
171/171 [==============================] - 47s 272ms/step - loss: 1.1169 - f1_score: 0.8153 - val_loss: 1.0722 - val_f1_score: 0.8301
Epoch 4/150
171/171 [==============================] - 46s 271ms/step - loss: 1.0398 - f1_score: 0.8422 - val_loss: 1.0653 - val_f1_score: 0.8296
Epoch 5/150
171/171 [==============================] - 47s 272ms/step - loss: 1.0166 - f1_score: 0.8523 - val_loss: 1.0145 - val_f1_score: 0.8500
Epoch 6/150
171/171 [==============================] - 47s 272ms/step - loss: 1.0004 - f1_score: 0.8580 - val_loss: 0.9884 - val_f1_score: 0.8612
Epoch 7/150
171/171 [==============================] - 47s 272ms/step - loss: 0.9854 - f1_score: 0.8646 - val_loss: 0.9854 - val_f1_scor

In [ ]:
model.save("RFF_512.h5")